In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import matplotlib.pyplot as plt
import urllib.request as urlreq

from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *

'''
Calculate the Black-Scholes implied volatility.

Parameters:	
price (float) – the Black-Scholes option price
S (float) – underlying asset price
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
>>> S = 100
>>> K = 100
>>> sigma = .2
>>> r = .01
>>> flag = 'c'
>>> t = .5
>>> price = black_scholes(flag, S, K, t, r, sigma)
>>> iv = implied_volatility(price, S, K, t, r, flag)
'''
%matplotlib inline

def write_excel(filename, sheetnames, df_list):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    for i, df in enumerate(df_list):
        
        df.to_excel(writer, sheet_name = sheetnames[i])

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    return

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [11]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    sp500.index.name = 'Tickers'
    return sp500

#nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#us_stocks = pd.concat([nyse,nasdaq], axis = 0).drop_duplicates()
#us_stocks = us_stocks[us_stocks['LastSale'] != 'n/a']
#us_fundamentals = get_fundas([string.replace(' ','') for string in us_stocks.index.tolist()])
us_stocks = pd.read_csv('us_stocks.csv', index_col = 0)

active_stocks = pd.read_csv('active_names.csv', index_col = 0).dropna()
active_etfs = pd.read_csv('active_etfs.csv', index_col = 0).dropna()
highest_ivs = pd.read_csv('highest_iv.csv', index_col = 0).dropna()

# filtered_names = pd.read_csv('filtered_names.csv', index_col = 0).join(us_stocks, how = 'inner')[us_stocks.columns]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(str).str[:-1]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(float)
# filtered_names = filtered_names.sort_values(['Market Cap'], ascending = False)

watchlist = ['NVDA', 'FB', 'AMZN', 'NFLX', 'GOOGL', 'GOOG',
             'TSLA', 'EA', 'ATVI', 'APPL', 'MSFT', 'INTC',
             'V', 'CSCO', 'VZ', 'T', 'MA', 'ORCL', 'IBM',
             'ADBE', 'TXN', 'AVGO', 'PYPL']

In [53]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.

def yahoo_table_parse(raw_html_table):
    tickers = []
    call_times = []
    implied_7_day = []
    implied_weekly = []
    eps = []
    i = 1
    end_row = 10
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        row_items = individual_row[0].split('</span>')[:3]

        if i == 1:
            i += 1
            continue
        tick = row_items[0].split('data-symbol="')[1].split('"')[0]
        os_check = optionslam_scrape(tick)

        if type(os_check) == str:
            continue
        else:
            tickers.append(tick)
            call_times.append(row_items[0].split('data-symbol="')[1].split('"')[-1].replace('>',''))
            eps.append(row_items[1].split('</td>')[1].split('>')[1])
            implied_7_day.append(os_check['Current 7 Day Implied'].replace('%',''))
            implied_weekly.append(os_check['Implied Move Weekly'].replace('%',''))


    return pd.DataFrame({'Tickers': tickers, 'Call Times': call_times, 'EPS': eps,
                         'Current 7 Day Implied': implied_7_day,
                         'Implied Move Weekly': implied_weekly})


def yahoo_earnings(date):
    # Yahoo Earnings Calendar Check

    today = date.strftime('%Y-%m-%d')
    tables = []
    for i in range(6):
        yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
        res = requests.get(yahoo_url)
        soup = bs(requests.get(yahoo_url).text, "lxml")

        try:
            table = soup.find_all('table')[0]
            tables.append(yahoo_table_parse(table))
        except:
            print('No Table')

    return pd.concat(tables,axis = 0, ignore_index = True)

'''
Functions for pulling options data from yahoo Input is a string. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
# Function for initial querying of yahoo data
def yahoo_option_query(ticker, unix_date):
    # dt.datetime.fromtimestamp(1525996800).date()
    if unix_date == 'None':
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}'.format(ticker)
    else:
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}?date={1}'.format(ticker,str(unix_date))
        
    response = urlreq.urlopen(yahoo_query)
    data = json.loads(response.read().decode())['optionChain']['result'][0]
    
    dict_lst = []
    for key in data.keys():
        dict_lst.append(data[key])
        
    expiries = dict_lst[1]
    strikes = dict_lst[2]
    underlying = dict_lst[4]
    calls = dict_lst[5][0]['calls']
    puts = dict_lst[5][0]['puts']
    
    return (expiries, strikes, underlying, calls, puts)

# Function for creating dataframe for options contracts for a specific maturity date
def create_contract_df(option_dict_lst, strikes):
    df = pd.DataFrame(columns = ['lastPrice','volume','openInterest','bid','ask','mid','impliedVolatility','expiration'],
                      index = strikes)
    for contract in option_dict_lst:
        for col in df.columns:
            if col == 'expiration':
                df.loc[contract['strike'], col] = (dt.datetime.fromtimestamp(contract['expiration']) - dt.datetime.today()).days
            elif col == 'mid':
                df.loc[contract['strike'], col] = (contract['ask'] + contract['bid'])/2
            else:
                df.loc[contract['strike'], col] = contract[col]
    return df.dropna()

# Function for creating straddle view of options for a specific date
def option_chain(calls, puts, strikes):
    call_contracts = create_contract_df(calls, strikes)
    put_contracts = create_contract_df(puts, strikes)
    return call_contracts.join(put_contracts, how = 'inner', lsuffix='_c', rsuffix='_p')

# Function for getting full option data for a specific ticker
def pull_options(ticker):
    initial_near_contract = yahoo_option_query(ticker, 'None')
    
    expiries = initial_near_contract[0]
    options_list = [option_chain(initial_near_contract[3], initial_near_contract[4], initial_near_contract[1])]
    
    for expiry in expiries[1:]:
        next_contract = yahoo_option_query(ticker, expiry)
        options_list.append(option_chain(next_contract[3], next_contract[4], next_contract[1]))
        
    return pd.concat(options_list, axis = 0)
ts.get_daily('AAPL')[0].tail(1)['close'][0]
'''
Function for getting all relevant earnings for a given starting week (Monday in dt.datetime(YYYY, m, d) format)
Returns a dataframe with the earnings names, implied move, price, and earnings times.
'''
def weekly_earnings_check(start_datetime, days_forward):

    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try:
            temp_earnings = yahoo_earnings(start_date)
            temp_earnings['Earnings Date'] = start_date
            temp_earnings['Last Close'] = 0
            for idx, row in temp_earnings.iterrows():
                temp_earnings.loc[idx, 'Last Close'] = ts.get_daily(row['Tickers'])[0].tail(1)['close'][0]
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    earnings_df = earnings_df[earnings_df['Last Close'] >= 30]
    earnings_df['Implied Move Weekly'] = pd.to_numeric(earnings_df['Implied Move Weekly'])
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    earnings_df['Lower Bound'] = np.round(earnings_df['Last Close']*(1 - earnings_df['Implied Move Weekly']/100),2)
    earnings_df = earnings_df.sort_values(['Earnings Date','Call Times'])
    
    return earnings_df

def stock_earnings(tick):
    yahoo_url = 'https://finance.yahoo.com/calendar/earnings/?symbol={0}'.format(tick)
    res = requests.get(yahoo_url).text
    res = res.split('table class')[1].split('</table>')[0].split('<tbody data-reactid="')[1].split('<td class="')
    earnings_cols = list(filter(lambda x: 'data-col2' in x, res))
    estimate_cols = list(filter(lambda x: 'data-col3' in x, res))
    reported_cols = list(filter(lambda x: 'data-col4' in x, res))

    earnings_days = []
    for i in earnings_cols:
        temp_date = i.split('">')[2].split('</')[0]
        temp_date = ''.join(temp_date.split(',')[:-1])
        temp_date = dt.datetime.strptime(temp_date, '%b %d %Y')
        earnings_days.append(temp_date)

    estimate_eps = []
    for i in estimate_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        estimate_eps.append(temp_est)

    reported_eps = []
    for i in reported_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        reported_eps.append(temp_est)

    earnings = pd.DataFrame({'Earnings Dates': earnings_days,
                             'EPS Estimate': estimate_eps,
                             'EPS Reported': reported_eps})
    return earnings

def old_weekly_earnings_check(start_datetime, days_forward):
    
    def yahoo_earnings_raw(raw_html_table):
        tickers = []
        eps = []
        i = 1
        end_row = 10
        for row in raw_html_table.find_all('tr'):
            # Individual row stores current row item and delimits on '\n'
            individual_row = str(row).split('\n')
            row_items = individual_row[0].split('</span>')[:3]

            if i == 1:
                i += 1
                continue
            tick = row_items[0].split('data-symbol="')[1].split('"')[0]
            tickers.append(tick)


        return pd.DataFrame({'Tickers': tickers})
    
    def yahoo_earnings_old(date):
        # Yahoo Earnings Calendar Check

        today = date.strftime('%Y-%m-%d')
        tables = []
        for i in range(6):
            yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
            res = requests.get(yahoo_url)
            soup = bs(requests.get(yahoo_url).text, "lxml")

            try:
                table = soup.find_all('table')[0]
                tables.append(yahoo_earnings_raw(table))
            except:
                print('No Table')

        return pd.concat(tables,axis = 0, ignore_index = True)
    
    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try: 
            temp_earnings = yahoo_earnings_old(start_date)
            temp_earnings['Earnings Date'] = start_date
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)
            continue

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    
    return earnings_df

    
def fundamentals(ticker):
    
    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan
        elif 'Market Cap' in row_items[0]:
            sum_item = 'Market Cap (B)'
            mkt_cap = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
            mkt_cap_amount = float(mkt_cap[:-1])
            if mkt_cap[-1] == 'M':
                sum_value = mkt_cap_amount/1000
            else:
                sum_value = mkt_cap_amount
        else:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
        
        sum_dict[sum_item] = sum_value
    
    sum_dict['Days Since Last Earnings'] = (dt.datetime.today().date() - 
                                            stock_earnings(ticker)['Earnings Dates'][1].date()).days

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        try:
            fund_lst.append(fundamentals(tick))
        except:
            continue
    return pd.concat(fund_lst,axis = 0)

# Function historical data from alpha advantage
def historical_data(ticker, day_number = 252, rolling_window = 20, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=rolling_window,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=rolling_window,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=rolling_window,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe.tail(day_number)

# Function for building a dataframe of volatilities
# Daily, Intraday, Overnight
def current_volatility(ticker_list, roll = 20):
    
    rows = []
    for tick in ticker_list:
        curr_vol = historical_data(tick).tail(1)[['daily_ann','intra_ann','ovrnt_ann','close',
                                                  'daily_dollar_vol']]
        curr_vol.index.name = 'Tickers'
        curr_vol.index = [tick]
        rows.append(curr_vol)
        
    return pd.concat(rows, axis = 0)

# Function for pulling S&P500 data and calculating volatilities
def sp500_filter():
    sp500 = pull_sp500_list()
    sp500_vols = current_volatility(sp500.index.tolist())
    df = pd.concat([sp500_vols, sp500], axis = 1).dropna()
    
    return df

def get_raw_data(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    return data

# Function for pulling options for a given ticker
def option_filter(ticker, moneyness_thresh, dte_thresh):
    data = get_raw_data(ticker).reset_index()
    data['Moneyness'] = np.abs(data['Strike'] - data['Underlying_Price'])/data['Underlying_Price']
    data['DTE'] = (data['Expiry'] - dt.datetime.today()).dt.days
    data = data[['Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 'Moneyness', 'Root', 'Underlying_Price',
                 'Last','Bid','Ask']]
    data['Mid'] = data['Ask'] - data['Bid']

    filtered_data = data[(data['Moneyness'] <= moneyness_thresh) &
                         (data['DTE'] <= dte_thresh)].reset_index()[data.columns]
    put_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', values='IV').dropna()
    call_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', values='IV').dropna()
    hv_data = current_volatility([ticker])

    put_ivs['Close'] = hv_data['close'][0]
    call_ivs['Close'] = hv_data['close'][0]
    put_ivs['Daily HV'] = hv_data['daily_ann'][0]
    call_ivs['Daily HV'] = hv_data['daily_ann'][0]
    put_ivs['Intra HV'] = hv_data['intra_ann'][0]
    call_ivs['Intra HV'] = hv_data['intra_ann'][0]
    put_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    call_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    put_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]
    call_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]

    call_ivs.index.name = ticker + ' Call Strike'
    put_ivs.index.name = ticker + ' Put Strike'
    return call_ivs, put_ivs


def phase1_options_filter(ticker_lst, hv_day_roll, atm_distance, dte_thresh):
    
    pulled_options = []
    calls = {}
    puts = {}

    moneyness_thresh = atm_distance

    for tick in ticker_lst:
        try:
            call, put = option_filter(tick, moneyness_thresh, dte_thresh)

            # Checking IV vs HV of the options chain out more than 2 weeks
            iv_chain_to_check = list(filter(lambda x: x >= hv_day_roll, 
                                            list(filter(lambda x: type(x) == int,call.columns.tolist()))))[0]
            average_difference_iv_hv = abs(call[iv_chain_to_check] - call['Intra HV']).mean()

            if average_difference_iv_hv >= 0.15:
                calls[tick] = call
                puts[tick] = put
                pulled_options.append(tick)
        except:
            pass
        
    return pulled_options, calls, puts

In [ ]:
tick_lst = active_stocks.head(100).index.tolist()
days_since_earnings_threshold = 30
mkt_cap_thresh = 5
moneyness_thresh = 0.05
hv_roll = 20
dte_thresh = 30
sort_on = 'ovrnt_ann'
max_options_num = 10

vols = current_volatility(tick_lst, roll = 20)
vols = vols.sort_values(['intra_ann'], ascending = False).dropna()
fundas = get_fundas(vols.index.tolist())
tickers = pd.concat([vols,fundas],axis = 1)
tickers = tickers[(tickers['Days Since Last Earnings'] >= days_since_earnings_threshold) &
                  (tickers['Market Cap (B)'] >= mkt_cap_thresh) &
                  (tickers['close'] >= 30)].sort_values([sort_on], ascending = False)

pulled_options, pulled_calls, pulled_puts = phase1_options_filter(tickers.index.tolist()[:max_options_num], 
                                                                  hv_roll, moneyness_thresh, dte_thresh)